In [ ]:
###############################################################################
# Machine Learning the 6th Dimension: 
# Stellar Radial Velocities from 5D Phase-Space Correlations
###############################################################################
# Edited by: Adriana Dropulic, Princeton University, 05-26-21
###############################################################################

In [ ]:
import keras
import tensorflow as tf
from keras import backend as K
from tensorflow.keras import initializers

In [ ]:
num_inputs = 5 #number of training variables
# Define training, validation, test sets
X_train = #shape is (nsamples, num_inputs)
y_train = 

X_val = 
y_val = 

X_test = 
y_test = 

# Define sample weights
# If including sample weights, they must be positive or loss will return nan
weights_train =
weights_val =

In [ ]:
def LikelihoodLossFunction(y_true, y_pred):
    # shape of y_pred should be (nsamples, 2)
    # the first column should be the line-of-sight velocity
    # the second column is the uncertainty
    SIGMA = K.abs(y_pred[:, 1]) + 1e-6
    LOC = y_pred[:, 0]
    X = y_true[:, 0]
    weights = y_true[:,1]
    ARG = K.pow((X - LOC),2) / (2 * K.pow(SIGMA,2))
    PREFACT = K.log(K.pow(2 * np.pi * K.pow(SIGMA,2), -0.5))
    return K.mean((ARG - PREFACT) * weights)


def ConstantLikelihoodLossFunction(y_true, y_pred):
    # shape of y_pred should be (nsamples, 2)
    # the first column should be the line-of-sight velocity
    # the second column is the uncertainty
    LOC = y_pred[:,0]
    X = y_true[:, 0]
    weights = y_true[:,1]
    ARG = K.square(X - LOC) / (2.0)
    PREFACT = K.log(K.pow(2 * np.pi, -0.5))
    return K.mean((ARG - PREFACT) * weights)

In [ ]:
#Include sample weights in y arrays
y_train = np.vstack([y_train, weights_train]).T
y_val = np.vstack([y_val, weights_val]).T

#Define the half-networks
initializer = tf.keras.initializers.glorot_uniform(seed=1)
activation = "tanh"
inputs = Input(shape=(num_inputs,))
nlayers = 

#Velocity predictor
MeanEst = (Dense(nlayers, activation=activation, kernel_initializer=initializer))(inputs)
MeanEst = (Dense(nlayers, activation=activation, kernel_initializer=initializer))(MeanEst)
MeanEst = (Dense(nlayers, activation=activation, kernel_initializer=initializer))(MeanEst)
MeanEst = (Dense(nlayers, activation=activation, kernel_initializer=initializer))(MeanEst)
MeanEst = (Dense(1, activation='linear', kernel_initializer=initializer))(MeanEst)
MeanModel = Model(inputs=[inputs], outputs=MeanEst)

#Uncertainty predictor
ConfEst= (Dense(nlayers, activation=activation, kernel_initializer=initializer))(inputs)
ConfEst= (Dense(nlayers, activation=activation, kernel_initializer=initializer))(ConfEst)
ConfEst= (Dense(nlayers, activation=activation, kernel_initializer=initializer))(ConfEst)
ConfEst= (Dense(nlayers, activation=activation, kernel_initializer=initializer))(ConfEst)
ConfEst= (Dense(1, activation='relu', kernel_initializer=initializer))(ConfEst)
ConfModel = Model(inputs=[inputs], outputs=ConfEst)

#Combined model
CombinedSub = Concatenate(axis=-1)([MeanModel(inputs), ConfModel(inputs)])
CombinedModel = Model(inputs=[inputs], outputs=CombinedSub)
CombinedModel.summary()

In [ ]:
#Define any callbacks
mycallbacks = 


In [ ]:
#Train the velocity predictor
ConfModel.trainable = False
MeanModel.trainable = True
CombinedModel.compile(loss=ConstantLikelihoodLossFunction,
                      optimizer='adam'
                     )
history = CombinedModel.fit(X_train,y_train,
                  validation_data=(X_val, y_val),
                  epochs=1000,
                  batch_size=10000,
                  callbacks = mycallbacks
                 )

In [ ]:
#Train the uncertainty predictor
ConfModel.trainable = True
MeanModel.trainable = False
CombinedModel.compile(loss=LikelihoodLossFunction,
                      optimizer='adam'
                     )

history = CombinedModel.fit(X_train,y_train,
                  validation_data=(X_val, y_val),
                  epochs=1000,
                  batch_size=10000,
                  callbacks = mycallbacks
                 )

In [ ]:
#Train both halves together to produce line-of-sight velocity and uncertainty value per star
ConfModel.trainable = True
MeanModel.trainable = True
CombinedModel.compile(loss=LikelihoodLossFunction,
                      optimizer='adam'
                     )

history = CombinedModel.fit(X_train,y_train,
                  validation_data=(X_val, y_val),
                  epochs=1000,
                  batch_size=10000,
                  callbacks = mycallbacks
                 )


In [ ]:
#Save the weights after completed training
CombinedModel.save_weights("ModelWeights.h5")

In [ ]:
#In separate notebook or file, can then do the following
CombinedModel.load_weights("ModelWeights.h5")
predictions = CombinedModel.predict(X_test)